# New Section

In [2]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as pd
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
import re

In [5]:
#from google.colab import files
#uploaded = files.upload()

In [7]:
df_train = pd.read_csv('Dataset/train.csv')
df_test = pd.read_csv('Dataset/test.csv')


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [9]:
df_train.drop('textID',axis=1,inplace=True)
df_train.drop('selected_text',axis=1,inplace=True)
df_train = df_train.dropna()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27480 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27480 non-null  object
dtypes: object(2)
memory usage: 644.1+ KB


In [10]:
df_test.drop('textID',axis=1,inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3534 non-null   object
 1   sentiment  3534 non-null   object
dtypes: object(2)
memory usage: 55.3+ KB


In [11]:
df = df_train.append(df_test)
#combining both the datasets as they were not in 80/20 ratio

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31014 entries, 0 to 3533
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       31014 non-null  object
 1   sentiment  31014 non-null  object
dtypes: object(2)
memory usage: 726.9+ KB


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'],test_size=0.2,random_state=42)

In [14]:
lemmatizer = WordNetLemmatizer() #changes word into its original meaning
stemmer = PorterStemmer()        #removes suffix
def smooth(text):
    
    #removing urls
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE)  # to remove links that start with HTTP/HTTPS in the tweet
    text = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE) # to remove other url links
    
    #removing apostrophes so that contracted words don't lose meaning and saving into remove_apos
    remove_apos = [word for word in text if word not in "'`"]
    remove_apos = ''.join(remove_apos)
    #print('Apostrophe removed: ',remove_apos)
    
    #removing punctuatin from remove_apos and saving into text_without_punc
    text_without_punc = [word if word not in punctuation else ' ' for word in remove_apos ]
    text_without_punc = ''.join(text_without_punc)
    #print('Without punctuation: ',text_without_punc)
    
    #removing stop words from text_without_punc and saving into text_without_sw
    text_without_sw = [word.lower() for word in text_without_punc.split() if word.lower() not in stopwords.words('english')]
    text_without_sw = ' '.join(text_without_sw)
    #print('Without stop words: ',text_without_sw)
    
    #removing numbers
    text_without_num = [word for word in text_without_sw.split() if not word.isdigit()]
    text_without_num = ' '.join(text_without_num)
    
    #lemmatizing text_without_sw and saving into lemmatized
    lemmatized = [lemmatizer.lemmatize(word) for word in text_without_num.split()]
    lemmatized = ' '.join(lemmatized)
    #print('Lemmatized: ',lemmatized)
    
    #stemming lemmatized and saving into clean_text
    clean_text = [stemmer.stem(word) for word in lemmatized.split()]
    stemmed = ' '.join(clean_text)
    #print('Stemmed: ', stemmed)
    return clean_text

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
bow = CountVectorizer(analyzer=smooth).fit(X_train,y_train)
bow_transformed = bow.transform(X_train)

In [6]:
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
  

True

In [16]:
tfidf = TfidfTransformer().fit(bow_transformed)
tfidf_transformed = tfidf.transform(bow_transformed)

In [17]:
#transforming testing set
bow_test = bow.transform(X_test)
bow_test_transformed = tfidf.transform(bow_test)  

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200).fit(tfidf_transformed, y_train)
rf.score(bow_test_transformed,y_test)

0.7107851039819442

In [23]:
pred_rf = rf.predict(bow_test_transformed)
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

    negative       0.73      0.60      0.66      1749
     neutral       0.66      0.74      0.70      2502
    positive       0.76      0.77      0.77      1952

    accuracy                           0.71      6203
   macro avg       0.72      0.70      0.71      6203
weighted avg       0.71      0.71      0.71      6203



In [26]:
from sklearn.svm import SVC
svc = SVC(C = 1000, gamma = 0.001).fit(tfidf_transformed, y_train)
svc.score(bow_test_transformed, y_test)

0.6945026600032242

In [28]:
pred_svc = svc.predict(bow_test_transformed)
print(classification_report(y_test, pred_svc))

              precision    recall  f1-score   support

    negative       0.68      0.63      0.65      1749
     neutral       0.65      0.71      0.68      2502
    positive       0.78      0.73      0.75      1952

    accuracy                           0.69      6203
   macro avg       0.70      0.69      0.69      6203
weighted avg       0.70      0.69      0.69      6203



In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(tfidf_transformed, y_train)
lr.score(bow_test_transformed, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.6916008383040464

In [31]:
pred_lr = lr.predict(bow_test_transformed)
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

    negative       0.71      0.59      0.64      1749
     neutral       0.63      0.75      0.69      2502
    positive       0.78      0.71      0.74      1952

    accuracy                           0.69      6203
   macro avg       0.71      0.68      0.69      6203
weighted avg       0.70      0.69      0.69      6203

